In [9]:
# Time series analysis of loudness over time
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

# Load master data and create time series plot
# Handle CSV parsing errors by using error_bad_lines=False and warn_bad_lines=True
df = pd.read_csv("master_data.csv", on_bad_lines='skip')

# Convert date column to datetime for proper time series plotting
df['date_formated_other'] = pd.to_datetime(df['date_formated_other'])

# Calculate daily averages for PLR norm
daily_avg = df.groupby(df['date_formated_other'].dt.date)['plr_norm'].mean().reset_index()
daily_avg['date_formated_other'] = pd.to_datetime(daily_avg['date_formated_other'])

# Calculate 3-day moving average
daily_avg = daily_avg.sort_values('date_formated_other')
daily_avg['plr_norm_3day_ma'] = daily_avg['plr_norm'].rolling(window=3, center=True, min_periods=1).mean()

# Create scatter plot without trendline
fig = px.scatter(
    df,
    x="date_formated_other",
    y="plr_norm",
    hover_name="Gong Master",
    custom_data=["timestamped_link"],  # Add URL data for click functionality
    title=""
)

# Add 3-day moving average line
fig.add_trace(go.Scatter(
    x=daily_avg['date_formated_other'],
    y=daily_avg['plr_norm_3day_ma'],
    mode='lines',
    name='3-Day Moving Average',
    line=dict(color='red', width=2),
    hovertemplate="3-Day Moving Average<br>%{x}<br>PLR: %{y:.2f}<extra></extra>",
    hoverlabel=dict(
        font=dict(family="gill sans", color="black"),
        bgcolor="white",
        bordercolor="red"
    )
))

# Style markers
fig.update_traces(
    marker=dict(color="black", size=5, opacity=.8, line=dict(width=0)),
    hovertemplate="<span style='font-weight: 600; font-family: gill sans;'>%{hovertext}</span><br>%{x}<br>PLR: %{y:.2f}<extra></extra>",
    hoverlabel=dict(
        font=dict(family="gill sans", color="black"),
        bgcolor="white",
        bordercolor="black"
    )
)

# Complete layout configuration
fig.update_layout(
    template="simple_white",
    font=dict(family="monotype bembo", size=14, color="black"),
    xaxis_title="",
    yaxis_title="Loudness",
    showlegend=False,
    legend=dict(
        x=0.02,
        y=0.98,
        bgcolor="white",
        bordercolor="black",
        borderwidth=1,
        font=dict(family="monotype bembo", size=12)
    ),
    width=647.2,
    height=400,
    margin=dict(l=10, r=10, t=10, b=10),
    title_font=dict(size=18),
    dragmode=False,
    xaxis=dict(
        showgrid=False,
        showline=True,
        zeroline=False,
        tickfont=dict(size=14),
        fixedrange=True,
        tickformat="%b %d"
    ),
    yaxis=dict(
        range=[.2, 1.05],
        tickmode='array',
        tickvals=[0.0, 0.2, 0.4, 0.6, 0.8, 1],
        ticktext=['0.0', '0.2', '.6', '.8', '1'],
        showgrid=False,
        showline=True,
        zeroline=False,
        tickfont=dict(size=14),
        fixedrange=True
    )
)

# Show chart with click functionality enabled
config = {
    "displaylogo": False, 
    "displayModeBar": False
}
fig.show(config=config)

# Write HTML with embedded JavaScript for click handling
html_string = fig.to_html(config=config, include_plotlyjs=True)

# Add JavaScript for click handling with debouncing
click_js = '''
<script>
document.addEventListener('DOMContentLoaded', function() {
    var clickTimeout = null;
    
    setTimeout(function() {
        var plotDiv = document.querySelector('.js-plotly-plot');
        if (plotDiv) {
            plotDiv.on('plotly_click', function(data) {
                // Check if a click is already being processed
                if (clickTimeout) {
                    return;
                }
                
                if (data.points && data.points[0] && data.points[0].customdata) {
                    var url = data.points[0].customdata[0];
                    if (url) {
                        // Set debounce timer to prevent rapid successive clicks
                        clickTimeout = setTimeout(function() {
                            clickTimeout = null;
                        }, 300);
                        
                        window.open(url, '_blank');
                    }
                }
            });
        }
    }, 100);
});
</script>
'''

# Insert JavaScript before closing body tag
html_with_clicks = html_string.replace('</body>', click_js + '</body>')

# Save the interactive HTML file
#with open("chart_all_scatter.html", "w") as f:
#    f.write(html_with_clicks)


/var/folders/wy/fdbsnc293115rtzg_wn8nh240000gn/T/ipykernel_26261/1619773785.py:11: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



In [ ]:
# Time series analysis of loudness over time
import plotly.express as px
import pandas as pd

# Load master data and create time series plot
# Handle CSV parsing errors by using error_bad_lines=False and warn_bad_lines=True
df = pd.read_csv("master_data.csv", on_bad_lines='skip')

# Convert date column to datetime for proper time series plotting
df['date_formated_other'] = pd.to_datetime(df['date_formated_other'])

# Create scatter plot without trendline
fig = px.scatter(
    df,
    x="date_formated_other",
    y="plr_norm",
    hover_name="Gong Master",
    custom_data=["timestamped_link"],  # Add URL data for click functionality
    title=""
)

# Style markers
fig.update_traces(
    marker=dict(color="black", size=5, opacity=.8, line=dict(width=0)),
    hovertemplate="<span style='font-weight: 600; font-family: gill sans;'>%{hovertext}</span><br>%{x}<br>PLR: %{y:.2f}<extra></extra>",
    hoverlabel=dict(
        font=dict(family="gill sans", color="black"),
        bgcolor="white",
        bordercolor="black"
    )
)

# Complete layout configuration
fig.update_layout(
    template="simple_white",
    font=dict(family="monotype bembo", size=14, color="black"),
    xaxis_title="",
    yaxis_title="Loudness",
    showlegend=False,
    width=647.2,
    height=400,
    margin=dict(l=10, r=10, t=10, b=10),
    title_font=dict(size=18),
    dragmode=False,
    xaxis=dict(
        showgrid=False,
        showline=True,
        zeroline=False,
        tickfont=dict(size=14),
        fixedrange=True,
        tickformat="%b %d"
    ),
    yaxis=dict(
        range=[.2, 1.05],
        tickmode='array',
        tickvals=[0.2, 0.4, 0.6, 0.8, 1],
        ticktext=['.2', '.4', '.6', '.8', '1'],
        showgrid=False,
        showline=True,
        zeroline=False,
        tickfont=dict(size=14),
        fixedrange=True
    )
)

# Show chart with click functionality enabled
config = {
    "displaylogo": False, 
    "displayModeBar": False
}
fig.show(config=config)

# Write HTML with embedded JavaScript for click handling
html_string = fig.to_html(config=config, include_plotlyjs=True)

# Add JavaScript for click handling with debouncing
click_js = '''
<script>
document.addEventListener('DOMContentLoaded', function() {
    var clickTimeout = null;
    
    setTimeout(function() {
        var plotDiv = document.querySelector('.js-plotly-plot');
        if (plotDiv) {
            plotDiv.on('plotly_click', function(data) {
                // Check if a click is already being processed
                if (clickTimeout) {
                    return;
                }
                
                if (data.points && data.points[0] && data.points[0].customdata) {
                    var url = data.points[0].customdata[0];
                    if (url) {
                        // Set debounce timer to prevent rapid successive clicks
                        clickTimeout = setTimeout(function() {
                            clickTimeout = null;
                        }, 300);
                        
                        window.open(url, '_blank');
                    }
                }
            });
        }
    }, 100);
});
</script>
'''

# Insert JavaScript before closing body tag
html_with_clicks = html_string.replace('</body>', click_js + '</body>')

# Save the interactive HTML file
#with open("chart_all_scatter.html", "w") as f:
#    f.write(html_with_clicks)


In [ ]:
# log funding variant
import plotly.express as px
import pandas as pd
import statsmodels.api as sm
import numpy as np
from sklearn.metrics import r2_score

df = pd.read_csv("funding_only.csv")

# Create log-transformed column for proper log-linear regression
df['log_funding'] = np.log10(df['Amount Raised'])

fig = px.scatter(
    df,
    x="log_funding",
    y="plr_norm",
    hover_name="Company",
    trendline="ols",
    title="Loudness vs Funding",
)

# Trendline style + hover (combined into single update)
fig.update_traces(
    selector=dict(mode="lines"),   # selects only the trendline traces
    line=dict(color="black", width=2, dash="dot"),  # customize style
    showlegend=True,  # make them appear in legend
    name="OLS Fit",
    hoverinfo="skip",  # completely disable hover for trendlines
    hovertemplate=None  # also set hovertemplate to None
)

fig.update_layout(
    template="simple_white",
    font=dict(family="Times New Roman", size=12, color="black"),
    xaxis_title="Funding Amount ($M)",
    yaxis_title="Loudness (PLR)",
    showlegend=False,
    width=700,
    height=500,
    margin=dict(l=40, r=20, t=40, b=40),
    xaxis=dict(
        tickmode='array',
        tickvals=[np.log10(3), np.log10(10), np.log10(30), np.log10(100), np.log10(300)],
        ticktext=['3', '10', '30', '100', '300'],
        showgrid=False,
        showline=True,
        zeroline=False,
    ),
    yaxis=dict(
        range=[0, 1],
        tickmode='array',
        tickvals=[0, .2, .4, .6, .8, 1],
        ticktext=['0', '.2', '.4', '.6', '.8', '1'],
        showgrid=False,
        showline=True,
        zeroline=False,
        
    ),
    title_font=dict(size=18),
    xaxis_tickfont=dict(size=14),
    yaxis_tickfont=dict(size=14),
)

# Scatter style + hover
fig.update_traces(
    selector=dict(mode="markers"),
    marker=dict(color="#016144",size=5, opacity=1, line=dict(width=0)),
    hovertemplate=
    "%{hovertext}<br>" +
    "Funding: $%{customdata:.0f}M<br>" +
    "PLR: %{y:.2f}<extra></extra>",
    customdata=df['Amount Raised']  # Use original funding values for hover
)

# r2 annotation calculation using log10 scaled funding
X_log = df['log_funding']  # Use the pre-calculated log column
y = df['plr_norm']
X_log = sm.add_constant(X_log)  # Add intercept
model = sm.OLS(y, X_log).fit()
r2 = model.rsquared

# r2 annotation
fig.add_annotation(
    xref="paper", 
    yref="paper",
    x=1.01, 
    y=0.8,
    text=f"R² = {r2:.3f}",
    showarrow=False,
    font=dict(
        size=12, 
        family="Times New Roman", 
        color="black"
    ),
)

# global design mechanics
fig.update_layout(
    dragmode=False,                 # disables drag-to-zoom, pan, lasso, etc.
    xaxis_fixedrange=True,          # lock x-axis range
    yaxis_fixedrange=True           # lock y-axis range
)

# modebar config
config = {
    "displaylogo": False,   # Plotly logo
    "modeBarButtonsToRemove": [
        "zoom2d",           # Zoom
        "pan2d",            # Pan
        "select2d",         # Box select
        "lasso2d",          # Lasso select
        "zoomIn2d",         # Zoom in
        "zoomOut2d",        # Zoom out
        "autoScale2d",      # Autoscale
        "resetScale2d",     # Reset axes
        "hoverClosestCartesian", # Closest point hover
        "hoverCompareCartesian", # Compare data hover
        "toggleSpikelines", # Spikelines
        "toImage",          # Download as PNG
        "sendDataToCloud",  # Deprecated cloud save
        "resetViews",       # Reset 3D views
        "orbitRotation",    # 3D orbit
        "tableRotation",    # 3D table rotation
        "zoom3d", "pan3d", "resetCameraDefault3d",
        "resetCameraLastSave3d", "hoverClosest3d",
        "toggleHover",      # Turn hover on/off
        "toggleHoverClosest", # (legacy alias)
        "toggleHoverCompare",
        "resetViews", "resetGeo", "resetViewMapbox"
    ],
    "displayModeBar": True,   # Always show
    # "displayModeBar": False # Never show
}

fig.show(config=config)


In [122]:
# the results
import plotly.express as px
import pandas as pd
import statsmodels.api as sm

# Load data and create scatter plot with color coding and trendlines
df = pd.read_csv("funding_only.csv")
fig = px.scatter(
    df,
    x="Amount Raised",
    y="plr_norm",
    color="Gong Master",
    hover_name="Company",
    custom_data=["timestamped_link"],  # Add URL data for click functionality
    trendline="ols",
    color_discrete_map={"John": "#016144", "Jordi": "#FF6B35"},
    title=""
)

# Style trendlines for each gong master
fig.update_traces(
    selector=dict(mode="lines", name="John"),
    line=dict(color="#016144", width=2, dash="dot"),
    hoverinfo="skip",
    hovertemplate=None,
    showlegend=False
)

fig.update_traces(
    selector=dict(mode="lines", name="Jordi"),
    line=dict(color="#d6584a", width=2, dash="dot"),
    hoverinfo="skip",
    hovertemplate=None,
    showlegend=False
)

# Style markers for each gong master
fig.update_traces(
    selector=dict(mode="markers", name="John"),
    marker=dict(color="#4C6A91", size=7, opacity=1, line=dict(width=0)),
    hovertemplate="<span style='font-weight: 600; font-family: gill sans;'>%{hovertext}</span><br>Funding: $%{x:.0f}M<br>PLR: %{y:.2f}<br>Gong Master: John<extra></extra>",
    hoverlabel=dict(
        font=dict(family="gill sans", color="black"),
        bgcolor="white",
        bordercolor="black"
    )
)

fig.update_traces(
    selector=dict(mode="markers", name="Jordi"),
    marker=dict(color="#D6584A", size=7, opacity=1, line=dict(width=0)),
    hovertemplate="<span style='font-weight: 600; font-family: gill sans;'>%{hovertext}</span><br>Funding: $%{x:.0f}M<br>PLR: %{y:.2f}<br>Gong Master: Jordi<extra></extra>",
    hoverlabel=dict(
        font=dict(family="gill sans", color="black"),
        bgcolor="white",
        bordercolor="black"
    )
)

# Calculate R² for each gong master
john_data = df[df['Gong Master'] == 'John']
jordi_data = df[df['Gong Master'] == 'Jordi']

X_john = sm.add_constant(john_data['Amount Raised'])
r2_john = sm.OLS(john_data['plr_norm'], X_john).fit().rsquared

X_jordi = sm.add_constant(jordi_data['Amount Raised'])
r2_jordi = sm.OLS(jordi_data['plr_norm'], X_jordi).fit().rsquared

# Complete layout configuration
fig.update_layout(
    template="simple_white",
    font=dict(family="monotype bembo", size=14, color="black"),
    xaxis_title="Funding Amount ($M)",
    yaxis_title="Loudness (PLR)",
    showlegend=False,
    legend=dict(
        x=0.02,
        y=0.98,
        bgcolor="rgba(255,255,255,0.8)",
        bordercolor="black",
        borderwidth=1,
        font=dict(family="gill sans", size=12)
    ),
    width=809,
    height=500,
    margin=dict(l=40, r=20, t=40, b=40),
    title_font=dict(size=18),
    dragmode=False,
    xaxis=dict(
        range=[0, 300],
        showgrid=False,
        showline=True,
        zeroline=False,
        tickfont=dict(size=14),
        fixedrange=True
    ),
    yaxis=dict(
        range=[.2, 1],
        tickmode='array',
        tickvals=[0.2, 0.4, 0.6, 0.8, 1],
        ticktext=['.2', '.4', '.6', '.8', '1'],
        showgrid=False,
        showline=True,
        zeroline=False,
        tickfont=dict(size=14),
        fixedrange=True
    )
)

# Add R² annotations for each gong master
fig.add_annotation(
    xref="paper", yref="paper",
    x=0.96, y=0.9,
    text=f"John R² = {r2_john:.3f}",
    showarrow=False,
    font=dict(size=12, family="gill sans", color="#016144")
)

fig.add_annotation(
    xref="paper", yref="paper",
    x=0.96, y=0.84,
    text=f"Jordi R² = {r2_jordi:.3f}",
    showarrow=False,
    font=dict(size=12, family="gill sans", color="#d6584a")
)

# Show chart with click functionality enabled
config = {
    "displaylogo": False, 
    "displayModeBar": False
}
fig.show(config=config)

# Write HTML with embedded JavaScript for click handling
html_string = fig.to_html(config=config, include_plotlyjs=True)

# Add JavaScript for click handling
click_js = '''
<script>
document.addEventListener('DOMContentLoaded', function() {
    setTimeout(function() {
        var plotDiv = document.querySelector('.js-plotly-plot');
        if (plotDiv) {
            plotDiv.on('plotly_click', function(data) {
                if (data.points && data.points[0] && data.points[0].customdata) {
                    var url = data.points[0].customdata[0];
                    if (url) {
                        window.open(url, '_blank');
                    }
                }
            });
        }
    }, 100);
});
</script>
'''

# Insert JavaScript before closing body tag
html_with_clicks = html_string.replace('</body>', click_js + '</body>')

# Save the interactive HTML file
with open("chart_2_hosts_split.html", "w") as f:
    f.write(html_with_clicks)



In [3]:
# John vs Jordi gong hits by Funding Stage
import pandas as pd
import plotly.graph_objects as go

# Load data and count by funding stage
df = pd.read_csv("funding_only.csv")
stages = ["Seed", "Series A", "Series B", "Series C", "Series D or Later"]

# Count data for John and Jordi
john_counts = df[df['Gong Master'] == 'John']['Funding Stage'].value_counts()
jordi_counts = df[df['Gong Master'] == 'Jordi']['Funding Stage'].value_counts()

# Create figure with predefined colors
colors = {"John": "#009966", "Jordi": "#ffe020"}
fig = go.Figure()

# Add bars with final styling
for name, counts, offset in [("John", john_counts, 0), ("Jordi", jordi_counts, 1)]:
    fig.add_trace(go.Bar(
        name=name,
        x=stages,
        y=[counts.get(stage, 0) for stage in stages],
        marker=dict(
            color=colors[name],
            line=dict(width=0.5, color="black"),
            opacity=1
        ),
        offsetgroup=offset,
        hoverinfo="none"
    ))

# Layout configuration
fig.update_layout(
    template="simple_white",
    font=dict(family="Times New Roman", size=12, color="black"),
    xaxis_title="",
    yaxis_title="Value",
    showlegend=True,
    width=647.2,
    height=400,
    margin=dict(l=40, r=20, t=50, b=60),
    xaxis=dict(
        showgrid=False,
        showline=True,
        linecolor="black",
        linewidth=1,
        zeroline=False,
        tickfont=dict(size=10),
        tickangle=-45
    ),
    yaxis=dict(
        range=[0, 20],
        tickmode='array',
        tickvals=list(range(0, 21, 2)),
        showgrid=False,
        showline=True,
        linecolor="black",
        linewidth=1,
        zeroline=False,
        tickfont=dict(size=12)
    ),
    plot_bgcolor="white",
    paper_bgcolor="white",
    legend=dict(
        x=0.8,
        y=0.9,
        bgcolor="rgba(255,255,255,0)",
        bordercolor="rgba(0,0,0,0)",
        font=dict(family="Times New Roman", size=10, color="black")
    )
)

# Show chart
fig.show(config={"displaylogo": False, "displayModeBar": False, "staticPlot": True})

#fig.write_html("chart_4_gong_hits_by_funding_stage.html")

# Save as PNG with high resolution
#fig.write_image("chart_4_funding_type_bar.png", 
#                width=647.2, 
#                height=400, 
#                scale=2)  # Higher scale for better quality